In [1]:
library(plyr)
library(sdmpredictors)
library(tidyterra)
library(tidyverse)
library(ggplot2)
library(raster)
library(terra)


#https://www.ecologi.st/spatial-r/raster-gis-operations-in-r-with-terra.html


Attaching package: 'tidyterra'


The following objects are masked from 'package:plyr':

    arrange, count, mutate, rename, summarise, summarize


The following object is masked from 'package:stats':

    filter


-- Attaching packages -------------------------------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.4.0      v purrr   0.3.5 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.1      v stringr 1.5.0 
v readr   2.1.3      v forcats 0.5.2 
-- Conflicts ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::arrange()   masks tidyterra::arrange(), plyr::arrange()
x purrr::compact()   masks plyr::compact()
x dplyr::count()     masks tidyterra::count(), plyr::count()
x dplyr::failwith()  masks plyr::failwit

In [2]:
(Bluecarbon_sites <- read.csv("data//BC/BCsampling_sites.tsv", sep ="\t", header=T) %>% 
                     dplyr::mutate(Site = ifelse(Site =="Site1", "Upper", "Lower")) %>%
                     dplyr::rename(estuary = Estuary) %>%
                     dplyr::rename(Biogeographical_region = Biogeographical.region))

estuary,Site,x,y,date,Biogeographical_region
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
Berg,Upper,18.19648,-32.81314,18 06 2021,Cool Temperate
Berg,Lower,18.14189,-32.77216,18 06 2021,Cool Temperate
Breede,Lower,20.84452,-34.39693,25 06 2021,Warm temperate
Breede,Upper,20.80405,-34.40552,25 06 2021,Warm temperate
Knysna,Upper,23.05105,-34.06119,30 05 2021,Warm temperate
Knysna,Lower,22.99216,-34.03369,30 05 2021,Warm temperate
Olifants,Lower,18.18882,-31.69302,05 07 2021,Cool Temperate
Olifants,Upper,18.19545,-31.66071,05 07 2021,Cool Temperate
Swartkops,Lower,25.59069,-33.83784,25 05 2021,Warm temperate


In [3]:
ZA_admin_st <- geodata::gadm("ZA", path = "data/sdm/")
ZA_admin_sf = sf::st_as_sf(ZA_admin_st)
ZA_extent <- raster::extent(10, 40, -37, -22)

In [4]:
get_raster <- function(path, pattern){

    file_ls <- Sys.glob(file.path(path , pattern))
    raster <- raster::stack(file_ls)
    
    return(raster)
} 

In [5]:
gmed_raster <- get_raster("GMED/*/", "*.asc")
dim(gmed_raster)

[1] 1784 4337   54

In [6]:
gmed_cropped <- raster::crop(gmed_raster, ZA_extent) 
dim(gmed_cropped)

[1] 181 362  54

In [7]:
as.data.frame(raster::extract(gmed_cropped, Bluecarbon_sites[c("x", "y")])) %>%
dplyr::select(where(function(x) any(is.na(x))))

bedtemp,bnitrate,bo2dissolve,bo2utilized,bphosphate,bsalin,bsalin21k,bsilicate,btemp,btemp21k,...,sst_mayoct,sst_novapr,sstmax,sstmean,sstmin,sstrange,surcurrent,tideaverage,waveheight,windspeed
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14.97160,6.56227,5.401511,0.369455,0.8635866,NA,NA,7.599373,NA,11.74407,...,NA,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,NA,4.207474,0.7635055
14.97160,6.56227,5.401511,0.369455,0.8635866,NA,34.95545,7.599373,NA,11.74407,...,16.04738,NA,16.98092,15.00000,13.69698,3.000000,0.009436653,0.1848701,4.207474,0.7635055
NA,NA,NA,NA,NA,NA,35.49804,NA,NA,18.02030,...,18.80169,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,35.49804,NA,NA,18.02030,...,18.80169,NA,NA,NA,NA,NA,NA,NA,NA,NA
15.56181,15.27581,4.562259,1.181571,0.8103373,NA,35.31949,10.791760,NA,16.51494,...,19.94017,19.9271,21.67366,18.67366,16.00000,4.923164,-0.011619920,0.8444477,7.000000,7.9088192
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
11.28881,16.46212,4.113858,2.060184,1.4847380,7.473438,34.93443,14.820160,NA,11.56553,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
11.28881,16.46212,4.113858,2.060184,1.4847380,7.473438,34.93443,14.820160,NA,11.56553,...,15.49737,NA,17.00000,15.00000,13.00000,3.650872,0.001738676,0.2083865,4.000000,0.7066119
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [8]:
get_approx_NA <-  function(xy, env_layer, max_dist){
    
    raster_dist <- replace(distanceFromPoints(env_layer, xy), is.na(env_layer), NA)
    dist_index <- which.min(raster_dist)
    dist_value <- raster_dist@data@values[dist_index]
    approx_value <- if(dist_value > max_dist) NA else  env_layer@data@values[dist_index]
    approx_df <- data.frame(x = xy['x'], y = xy['y'], dist_value, approx_value)
    names(approx_df)[3:4] <- c(paste('dist', names(env_layer), sep = '_'), names(env_layer))
        
return(approx_df)  
}

In [9]:
proxim_var <- function(layer_name, raster_stack, max_dist){
    
    env_layer <- raster_stack[[layer_name]]
    proxim_na <- do.call(rbind, apply(X = coord, MARGIN = 1, FUN = get_approx_NA, env_layer, max_dist))

return(proxim_na)
}

In [10]:
proxim_df_ls <- lapply(names(gmed_cropped), proxim_var, raster_stack = gmed_cropped, max_dist = 10000)
proxim_df <- join_all(proxim_values, by = c('x','y'), type ="full")

ERROR: Error in apply(X = coord, MARGIN = 1, FUN = get_approx_NA, env_layer, : object 'coord' not found


In [ ]:
proxim_df

In [ ]:
fill_envdata <- function(my_layer, env_layers, coord_df, proxim_data){
    
    env_layer <- env_layers[[my_layer]]
    for (i in 1: nrow(Bluecarbon_sites)){
        xy <- coord_df[i,c('x','y')]
        env_value <- raster::extract(env_layer, xy)
        if(is.na(env_value) & !missing(proxim_data)){
            proxim_value <- proxim_df[proxim_df$x == coord_df[i,]$x & proxim_df$y ==  coord_df[i,]$y, my_layer]
            coord_df[i, my_layer] <- proxim_value
        }
        else{
            coord_df[i, my_layer] <- env_value  
        }
    }   
return(coord_df)
}

In [ ]:
filled_env_data <- lapply(names(gmed_cropped), fill_envdata, env_layers = gmed_cropped, coord_df = Bluecarbon_sites, proxim_data = proxim_df)

In [ ]:
proxim_df <- join_all(filled_env_data, by = c("estuary","Site","x","y","date","Biogeographical_region"), type ="full")

In [ ]:
proxim_df

In [ ]:
#Testing filling of NAs 
#https://stackoverflow.com/questions/27562076/if-raster-value-na-search-and-extract-the-nearest-non-na-pixel
set.seed(2)
# create a 10x10 raster
r <- raster(ncol=10,nrow=10, xmn=0, xmx=10, ymn=0,ymx=10)
r[] <- 1:10
r[sample(1:ncell(r), size = 25)] <- NA
# plot the raster
plot(r, axes=F, box=F)
segments(x0 = 0, y0 = 0:10, x1 = 10, y1 = 0:10, lty=2)
segments(y0 = 0, x0 = 0:10, y1 = 10, x1 = 0:10, lty=2)
# create sample points and add them to the plot
xy = data.frame(x=runif(10,1,10), y=runif(10,1,10))
points(xy, pch=3)
text(x = xy$x, y = xy$y, labels = as.character(1:nrow(xy)), pos=4, cex=0.7, xpd=NA)

In [ ]:
sampled = apply(X = xy, MARGIN = 1, FUN = function(xy) r@data@values[which.min(replace(distanceFromPoints(r, xy), is.na(r), NA))])
extracted = extract(x = r, y = xy)
sampled
print(data.frame(xy, extracted, sampled))

In [ ]:
#testing the implementation
do.call(rbind, apply(X = xy, MARGIN = 1, FUN = get_approx_NA, env_layer = r, max_dist = 1000000))